In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/mlproj/train.csv")
xtest=pd.read_csv("/content/drive/MyDrive/mlproj/test.csv")

In [ ]:
df.head(5)

In [ ]:
pd.set_option('display.float_format',  '{:,}'.format)
pd.set_option('display.max_columns', None)
df.describe()


In [ ]:
df.info()
# total features=83
# dtypes: float64(36), int64(18), object(29)
# no. of rows =1427437

In [ ]:
# checking if the target column is balanced or biased

In [ ]:
print(df["HasDetections"].value_counts())
plt.hist(df["HasDetections"]);
# both 0 and 1 values occur around 7 lakh times....so the target is Balanced

In [ ]:
# some 200 duplicate rows in the df
df=df.drop_duplicates()


In [ ]:
# df.fillna(np.nan,inplace=True)

In [ ]:
pd.set_option('display.max_rows', None)
describem = []
for col in df.columns:
    describem.append((col, df[col].isnull().sum() * 100 / df.shape[0],df[col].dtype))   
describem = pd.DataFrame(describem, columns=['feature', '% missing values','type'])
describem.sort_values('% missing values', ascending=False)

In [ ]:
xtest.info()

In [ ]:
# missing value percentage >70 percent====removal

highmiss=['PuaMode','Census_ProcessorClass','DefaultBrowsersIdentifier','Census_IsFlightingInternal','Census_InternalBatteryType']
df.drop(columns=highmiss,inplace=True)
xtest.drop(columns=highmiss,inplace=True)



In [ ]:
df.shape

In [ ]:
describes = []
for col in df.columns:
    describes.append((col, df[col].value_counts(dropna=False).values[0] * 100/df.shape[0],df[col].dtype))   
desc = pd.DataFrame(describes, columns=['feature', 'skewness','type'])
desc.sort_values('skewness', ascending=False)

In [ ]:
# 6                                                IsBeta
# 27                                      AutoSampleOptIn
# 62                     Census_IsPortableOperatingSystem
# 34                                  Census_DeviceFamily
# 32                                         UacLuaenable
# 72                               Census_IsVirtualDevice
# 2                                           ProductName
# 12                                               HasTpm
# 8                                      IsSxsPassiveMode
# 65                             Census_IsFlightsDisabled
# 11                                    AVProductsEnabled
# 7                                      RtpStateBitfield
# 31                                             Firewall
# 20                                                OsVer
# 18                                             Platform
# 74                                  Census_IsPenCapable
# 26                                          IsProtected
# 28                                                SMode
# 66                                    Census_FlightRing
# 75              Census_IsAlwaysOnAlwaysConnectedCapable
# 43                           Census_HasOpticalDiskDrive
# 52                                Census_OSArchitecture
# 19                                            Processor




# these cols have high skewness(>90%)
# therefore removing them,except for      isprotected,  firewall,   smode



In [ ]:
highskew=[                         'IsBeta',
                                      'AutoSampleOptIn',
                     'Census_IsPortableOperatingSystem',
                                  'Census_DeviceFamily',
                                         'UacLuaenable',
                               'Census_IsVirtualDevice',
                                           'ProductName',
                                               'HasTpm',
                                      'IsSxsPassiveMode',
                             'Census_IsFlightsDisabled',
                                    'AVProductsEnabled',
                                      'RtpStateBitfield',
                                             
                                                'OsVer',
                                             'Platform',
                                  'Census_IsPenCapable',
                                          
                                                
                                    'Census_FlightRing',
              'Census_IsAlwaysOnAlwaysConnectedCapable',
                           'Census_HasOpticalDiskDrive',
                                'Census_OSArchitecture',
                                           'Processor']
df.drop(columns=highskew,inplace=True)
xtest.drop(columns=highskew,inplace=True)
df.shape

In [ ]:
df["Unnamed: 0"].corr(df["MachineIdentifier"])

In [ ]:
df.drop(columns="Unnamed: 0",inplace=True)
xtest.drop(columns="Unnamed: 0",inplace=True)
  

In [ ]:
# each pc is identified by a unique machine id
df["MachineIdentifier"].isnull().sum()

In [ ]:
stat = []
for col in df.columns:
    stat.append((col,df[col].isnull().sum() * 100 / df.shape[0] ,df[col].value_counts(dropna=False).values[0] * 100/df.shape[0],df[col].dtype))   
stat = pd.DataFrame(stat, columns=['feature','null', 'skewness','type'])
stat.sort_values('null', ascending=False)

In [ ]:
plt.figure(figsize=(100,50))
cor = df.corr()
sns.heatmap(cor, cmap="Blues", annot=True)
plt.show()

In [ ]:
# firstly the OsBuild and and censusOSbuildNumber has hight corr with otne other(0.94)
# going to keep censusOSbuildNumber and droping OsBuild
df.drop(columns="OsBuild",inplace=True)
xtest.drop(columns="OsBuild",inplace=True)

In [ ]:
print(df.groupby("HasDetections")["Firewall"].value_counts())
# as shown......firewall==0 or 1 has no significant impact on detections
print(df["Firewall"].corr(df["HasDetections"]))
# correlation is also weak
# skewness is >90%


In [ ]:
print((df["Firewall"].isna().sum()/df.shape[0])*100)


In [ ]:
df.groupby("Firewall",dropna=False)["HasDetections"].value_counts()

In [ ]:
# as first we thought may be firewall is directly linked to the detections...but it has everything agaist it...like low corr(near zero)....high skewness
df.drop(columns="Firewall",inplace=True)
xtest.drop(columns="Firewall",inplace=True)

In [ ]:
print(df["IsProtected"].isna().sum())
print(xtest["IsProtected"].isna().sum())
# null values in df and xtest of the feature

In [ ]:
df["IsProtected"].value_counts()
# skewed

In [ ]:
df.groupby(by="IsProtected",dropna=False)["HasDetections"].value_counts()

In [ ]:
# when isprotected is 0.....detections should have been more....but it turns out detections are less
df["IsProtected"].fillna(0).corr(df["HasDetections"])


In [ ]:
df["IsProtected"].fillna(0,inplace=True)
xtest["IsProtected"].fillna(0,inplace=True)

In [ ]:
df["SMode"].value_counts()

In [ ]:
df["SMode"].isnull().sum()

In [ ]:
df.groupby("SMode",dropna=False)["HasDetections"].value_counts()

In [ ]:
# when Smode is on----------->Detections are comparetively low

In [ ]:
xtest["SMode"].value_counts(dropna=False)

In [ ]:

df["SMode"].fillna(1).corr(df["HasDetections"])

In [ ]:
df["SMode"].fillna(1,inplace=True)
xtest["SMode"].fillna(1,inplace=True)

In [ ]:
df['Census_ThresholdOptIn'].value_counts(dropna=False)
# majority null values
# correlation is really weak(0.0009)
# the feature is not explained(function or description of the feature) in the given dataset as well 

In [ ]:
df.groupby("Census_ThresholdOptIn",dropna=False)["HasDetections"].value_counts()

In [ ]:
df['Census_ThresholdOptIn'].fillna(0).corr(df["HasDetections"])

In [ ]:
df['Census_ThresholdOptIn'].fillna(1).corr(df["HasDetections"])
# if null values are filled with 1...correlation increases a bit.
# for now filling with 1....


In [ ]:
df['Census_ThresholdOptIn'].fillna(1,inplace=True)
xtest['Census_ThresholdOptIn'].fillna(1,inplace=True)

## Categorical Data

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
df["SmartScreen"].value_counts()

In [ ]:
df["SmartScreen"]=df["SmartScreen"].replace("off","Off")
xtest["SmartScreen"]=xtest["SmartScreen"].replace("off","Off")

In [ ]:
df["SmartScreen"]=df["SmartScreen"].replace("prompt","Prompt")
xtest["SmartScreen"]=xtest["SmartScreen"].replace("prompt","Prompt")

In [ ]:
df["SmartScreen"]=df["SmartScreen"].replace("requireadmin","RequireAdmin")
xtest["SmartScreen"]=xtest["SmartScreen"].replace("requireadmin","RequireAdmin")

In [ ]:
df["SmartScreen"]=df["SmartScreen"].replace("on","On")
xtest["SmartScreen"]=xtest["SmartScreen"].replace("on","On")

In [ ]:
df["SmartScreen"].value_counts(dropna=False)

In [ ]:
df.groupby("SmartScreen",dropna=False)["HasDetections"].value_counts()

In [ ]:
# existsNotSet if 1 gives higher detections
# Nan value has equal detections and no detections
xtest["SmartScreen"].value_counts(dropna=False)

In [ ]:
print(pd.Series(le.fit_transform(df["SmartScreen"].fillna("RequireAdmin"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["SmartScreen"].fillna("ExistsNotSet"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["SmartScreen"].fillna("Off"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["SmartScreen"].fillna("Warn"))).corr(df["HasDetections"]))

In [ ]:
df["SmartScreen"].fillna("RequireAdmin",inplace=True)
xtest["SmartScreen"].fillna("RequireAdmin",inplace=True)

In [ ]:
df["Census_PrimaryDiskTypeName"].value_counts(dropna=False)

In [ ]:
df.groupby("Census_PrimaryDiskTypeName",dropna=False)["HasDetections"].value_counts()

In [ ]:
# going with the idea that this dataset is supposed to be collected some years ago...so HDD would be more common
# the detections are almost balanced in all types of disks

In [ ]:
print(pd.Series(le.fit_transform(df["Census_PrimaryDiskTypeName"].fillna("HDD"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["Census_PrimaryDiskTypeName"].fillna("SSD"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["Census_PrimaryDiskTypeName"].fillna("UNKNOWN"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["Census_PrimaryDiskTypeName"].fillna("Unspecified"))).corr(df["HasDetections"]))

In [ ]:
df["Census_PrimaryDiskTypeName"].fillna("HDD",inplace=True)
df["Census_PrimaryDiskTypeName"].fillna("HDD",inplace=True)

In [ ]:

df.groupby("Census_ChassisTypeName",dropna=False)["HasDetections"].value_counts()

In [ ]:
print(pd.Series(le.fit_transform(df["Census_ChassisTypeName"].fillna("Notebook"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["Census_ChassisTypeName"].fillna("Laptop"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["Census_ChassisTypeName"].fillna("Desktop"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["Census_ChassisTypeName"].fillna("Other"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["Census_ChassisTypeName"].fillna("Portable"))).corr(df["HasDetections"]))


In [ ]:
df["Census_ChassisTypeName"].value_counts()

In [ ]:
# as notebook appears most of the times...filling na with mode
df["Census_ChassisTypeName"].fillna("Notebook",inplace=True)
xtest["Census_ChassisTypeName"].fillna("Notebook",inplace=True)

In [ ]:
df["Census_PowerPlatformRoleName"].value_counts(dropna=False)

In [ ]:
df["Census_PowerPlatformRoleName"].isna().sum()

In [ ]:
# only 10 na values
# therefore filling with ffill
df["Census_PowerPlatformRoleName"].fillna(method="ffill",limit=3,inplace=True)
xtest["Census_PowerPlatformRoleName"].fillna(method="ffill",limit=3,inplace=True)

In [ ]:
print(df["Census_PowerPlatformRoleName"].isna().sum())
print(xtest["Census_PowerPlatformRoleName"].isna().sum())





In [ ]:
df["OsBuildLab"].value_counts(dropna=False)

In [ ]:
# df.groupby("OsBuildLab",dropna=False)["HasDetections"].value_counts()

In [ ]:
# 17134.1.amd64fre.rs4_release.180410-1804                     584858
# 16299.431.amd64fre.rs3_release_svc_escrow.180502-1908        200554
# 16299.15.amd64fre.rs3_release.170928-1534                    154078
# 15063.0.amd64fre.rs2_release.170317-1834                     114569
print(pd.Series(le.fit_transform(df["OsBuildLab"].fillna("17134.1.amd64fre.rs4_release.180410-1804"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["OsBuildLab"].fillna("16299.431.amd64fre.rs3_release_svc_escrow.180502-1908"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["OsBuildLab"].fillna("16299.15.amd64fre.rs3_release.170928-1534"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["OsBuildLab"].fillna("15063.0.amd64fre.rs2_release.170317-1834"))).corr(df["HasDetections"]))
print(pd.Series(le.fit_transform(df["OsBuildLab"].fillna("17134.1.x86fre.rs4_release.180410-1804"))).corr(df["HasDetections"]))


In [ ]:
df["OsBuildLab"].fillna("17134.1.amd64fre.rs4_release.180410-1804",inplace=True)
xtest["OsBuildLab"].fillna("17134.1.amd64fre.rs4_release.180410-1804",inplace=True)

#NUMERIC DATA

In [ ]:
# for now ,we followed the strategy:
# if distribution is uniform:::replace nan by median
# if distribution is skewed:::: replace by mode

In [ ]:
df.groupby("Census_IsWIMBootEnabled",dropna=False)["HasDetections"].value_counts()
# uniformly distributed detections
# null values are large

In [ ]:
df.drop(columns="Census_IsWIMBootEnabled",inplace=True)
xtest.drop(columns="Census_IsWIMBootEnabled",inplace=True)

In [ ]:
plt.hist(df['Wdft_RegionIdentifier'])
plt.show() 

In [ ]:
sns.boxplot(df['Wdft_RegionIdentifier']);

In [ ]:

sns.scatterplot(x="Wdft_RegionIdentifier", y="HasDetections", data=df);

In [ ]:
df["Wdft_RegionIdentifier"].fillna(df.groupby("HasDetections")["Wdft_RegionIdentifier"].transform("median"),inplace=True)
xtest["Wdft_RegionIdentifier"].fillna(df["Wdft_RegionIdentifier"].median(),inplace=True)

In [ ]:
df["Wdft_RegionIdentifier"].corr(df["HasDetections"])

In [ ]:
plt.hist(df['Census_SystemVolumeTotalCapacity'])
plt.show() 


In [ ]:
df['Census_SystemVolumeTotalCapacity'].corr(df["HasDetections"])

In [ ]:
sns.boxplot(df['Census_SystemVolumeTotalCapacity']);

In [ ]:
sns.scatterplot(x="Census_SystemVolumeTotalCapacity", y="HasDetections", data=df);

In [ ]:
df['Census_SystemVolumeTotalCapacity'].fillna(df['Census_SystemVolumeTotalCapacity'].mode()[0],inplace=True)
xtest['Census_SystemVolumeTotalCapacity'].fillna(df['Census_SystemVolumeTotalCapacity'].mode()[0],inplace=True)


In [ ]:
plt.hist(df['IeVerIdentifier'])
plt.show() 

In [ ]:
sns.scatterplot(df["IeVerIdentifier"], df["HasDetections"]);

In [ ]:
sns.boxplot(df["IeVerIdentifier"])

In [ ]:
df['IeVerIdentifier'].fillna(df['IeVerIdentifier'].mode()[0],inplace=True)
xtest['IeVerIdentifier'].fillna(df['IeVerIdentifier'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['Census_OEMNameIdentifier'])
plt.show() 

In [ ]:
df['Census_OEMNameIdentifier'].fillna(df['Census_OEMNameIdentifier'].median(),inplace=True)
xtest['Census_OEMNameIdentifier'].fillna(df['Census_OEMNameIdentifier'].median(),inplace=True)

In [ ]:
plt.hist(df['Census_OEMModelIdentifier'])
plt.show() 

In [ ]:
df['Census_OEMModelIdentifier'].fillna(df['Census_OEMModelIdentifier'].median(),inplace=True)
xtest['Census_OEMModelIdentifier'].fillna(df['Census_OEMModelIdentifier'].median(),inplace=True)

In [ ]:
plt.hist(df['Census_FirmwareManufacturerIdentifier'])
plt.show() 

In [ ]:
df['Census_FirmwareManufacturerIdentifier'].fillna(df['Census_FirmwareManufacturerIdentifier'].median(),inplace=True)
xtest['Census_FirmwareManufacturerIdentifier'].fillna(df['Census_FirmwareManufacturerIdentifier'].median(),inplace=True)

In [ ]:
plt.hist(df['Wdft_IsGamer'])
plt.show() 

In [ ]:
df['Wdft_IsGamer'].fillna(df['Wdft_IsGamer'].mode()[0],inplace=True)
xtest['Wdft_IsGamer'].fillna(df['Wdft_IsGamer'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['Census_InternalBatteryNumberOfCharges'])
plt.show()

In [ ]:
df['Census_InternalBatteryNumberOfCharges'].fillna(df['Census_InternalBatteryNumberOfCharges'].mode()[0],inplace=True)
xtest['Census_InternalBatteryNumberOfCharges'].fillna(df['Census_InternalBatteryNumberOfCharges'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['Census_OSInstallLanguageIdentifier'])
plt.show() 

In [ ]:
df['Census_OSInstallLanguageIdentifier'].fillna(df['Census_OSInstallLanguageIdentifier'].mode()[0],inplace=True)
xtest['Census_OSInstallLanguageIdentifier'].fillna(df['Census_OSInstallLanguageIdentifier'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['Census_TotalPhysicalRAM'])
plt.show()

In [ ]:
df['Census_TotalPhysicalRAM'].fillna(df['Census_TotalPhysicalRAM'].mode()[0],inplace=True)
xtest['Census_TotalPhysicalRAM'].fillna(df['Census_TotalPhysicalRAM'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['OrganizationIdentifier'])
plt.show() 

In [ ]:
df['OrganizationIdentifier'].fillna(df['OrganizationIdentifier'].mode()[0],inplace=True)
xtest['OrganizationIdentifier'].fillna(df['OrganizationIdentifier'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['CityIdentifier'])
plt.show() 

In [ ]:
df['CityIdentifier'].fillna(df['CityIdentifier'].median(),inplace=True)
xtest['CityIdentifier'].fillna(df['CityIdentifier'].median(),inplace=True)

In [ ]:
plt.hist(df['Census_InternalBatteryNumberOfCharges'])
plt.show()

In [ ]:
df['Census_InternalBatteryNumberOfCharges'].fillna(df['Census_InternalBatteryNumberOfCharges'].mode()[0],inplace=True)
xtest['Census_InternalBatteryNumberOfCharges'].fillna(df['Census_InternalBatteryNumberOfCharges'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['Census_FirmwareVersionIdentifier'])
plt.show() 

In [ ]:
df['Census_FirmwareVersionIdentifier'].fillna(df['Census_FirmwareVersionIdentifier'].median(),inplace=True)
xtest['Census_FirmwareVersionIdentifier'].fillna(df['Census_FirmwareVersionIdentifier'].median(),inplace=True)

In [ ]:
plt.hist(df['Census_FirmwareVersionIdentifier'])
plt.show()

In [ ]:
df['Census_FirmwareVersionIdentifier'].fillna(df['Census_FirmwareVersionIdentifier'].median(),inplace=True)
xtest['Census_FirmwareVersionIdentifier'].fillna(df['Census_FirmwareVersionIdentifier'].median(),inplace=True)

In [ ]:
plt.hist(df['Census_OEMNameIdentifier'])
plt.show() 

In [ ]:
df['Census_OEMNameIdentifier'].fillna(df['Census_OEMNameIdentifier'].median(),inplace=True)
xtest['Census_OEMNameIdentifier'].fillna(df['Census_OEMNameIdentifier'].median(),inplace=True)

In [ ]:
plt.hist(df['IeVerIdentifier'])
plt.show() 

In [ ]:
df['IeVerIdentifier'].fillna(df['IeVerIdentifier'].mode()[0],inplace=True)
xtest['IeVerIdentifier'].fillna(df['IeVerIdentifier'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['Census_PrimaryDiskTotalCapacity'])
plt.show()

In [ ]:
df['Census_PrimaryDiskTotalCapacity'].fillna(df['Census_PrimaryDiskTotalCapacity'].mode()[0],inplace=True)
xtest['Census_PrimaryDiskTotalCapacity'].fillna(df['Census_PrimaryDiskTotalCapacity'].mode()[0],inplace=True)



In [ ]:
plt.hist(df['Census_SystemVolumeTotalCapacity'])
plt.show() 

In [ ]:
df['Census_SystemVolumeTotalCapacity'].fillna(df['Census_SystemVolumeTotalCapacity'].mode()[0],inplace=True)
xtest['Census_SystemVolumeTotalCapacity'].fillna(df['Census_SystemVolumeTotalCapacity'].mode()[0],inplace=True)


In [ ]:
plt.hist(df['Census_OEMNameIdentifier'])
plt.show() 

In [ ]:
df['Census_OEMNameIdentifier'].fillna(df['Census_OEMNameIdentifier'].median(),inplace=True)
xtest['Census_SystemVolumeTotalCapacity'].fillna(df['Census_SystemVolumeTotalCapacity'].mode()[0],inplace=True)


In [ ]:
plt.hist(df['Census_InternalPrimaryDisplayResolutionHorizontal'])
plt.show()

In [ ]:
df['Census_InternalPrimaryDiagonalDisplaySizeInInches'].fillna(df['Census_InternalPrimaryDiagonalDisplaySizeInInches'].mode()[0],inplace=True)
xtest['Census_InternalPrimaryDiagonalDisplaySizeInInches'].fillna(df['Census_InternalPrimaryDiagonalDisplaySizeInInches'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['Census_InternalPrimaryDisplayResolutionHorizontal'])
plt.show() 

In [ ]:
df['Census_InternalPrimaryDisplayResolutionHorizontal'].fillna(df['Census_InternalPrimaryDisplayResolutionHorizontal'].mode()[0],inplace=True)
xtest['Census_InternalPrimaryDisplayResolutionHorizontal'].fillna(df['Census_InternalPrimaryDisplayResolutionHorizontal'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['Census_InternalPrimaryDisplayResolutionVertical'])
plt.show() 

In [ ]:
df['Census_InternalPrimaryDisplayResolutionVertical'].fillna(df['Census_InternalPrimaryDisplayResolutionVertical'].mode()[0],inplace=True)
xtest['Census_InternalPrimaryDisplayResolutionVertical'].fillna(df['Census_InternalPrimaryDisplayResolutionVertical'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['Census_ProcessorModelIdentifier'])
plt.show() 

In [ ]:
df['Census_ProcessorModelIdentifier'].fillna(df['Census_ProcessorModelIdentifier'].median(),inplace=True)
xtest['Census_ProcessorModelIdentifier'].fillna(df['Census_ProcessorModelIdentifier'].median(),inplace=True)

In [ ]:
plt.hist(df['Census_ProcessorManufacturerIdentifier'])
plt.show() 

In [ ]:
df['Census_ProcessorManufacturerIdentifier'].fillna(df['Census_ProcessorManufacturerIdentifier'].mode()[0],inplace=True)
xtest['Census_ProcessorManufacturerIdentifier'].fillna(df['Census_ProcessorManufacturerIdentifier'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['Census_ProcessorCoreCount'])
plt.show()

In [ ]:
df['Census_ProcessorCoreCount'].fillna(df['Census_ProcessorCoreCount'].mode()[0],inplace=True)
xtest['Census_ProcessorCoreCount'].fillna(df['Census_ProcessorCoreCount'].mode()[0],inplace=True)



In [ ]:
plt.hist(df['AVProductStatesIdentifier'])
plt.show() 

In [ ]:
df['AVProductStatesIdentifier'].fillna(df['AVProductStatesIdentifier'].mode()[0],inplace=True)
xtest['AVProductStatesIdentifier'].fillna(df['AVProductStatesIdentifier'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['AVProductsInstalled'])
plt.show() 

In [ ]:
df['AVProductsInstalled'].fillna(df['AVProductsInstalled'].mode()[0],inplace=True)
xtest['AVProductsInstalled'].fillna(df['AVProductsInstalled'].mode()[0],inplace=True)

In [ ]:
plt.hist(df['GeoNameIdentifier'])
plt.show() 

In [ ]:
df['GeoNameIdentifier'].fillna(df['GeoNameIdentifier'].median(),inplace=True)
xtest['GeoNameIdentifier'].fillna(df['GeoNameIdentifier'].median(),inplace=True)

In [ ]:
columns=[]
for col in df.columns:
  columns.append(col)

In [ ]:
dfm1=df

In [ ]:
#  encodng

In [ ]:
le=LabelEncoder()
df2=df.select_dtypes(include='object')

In [ ]:
for col in df2.columns:
     dfm1[col]=le.fit_transform(dfm1[col])

In [ ]:
cor={}
for col in columns:
  cor[col]=abs(df[col].corr(df["HasDetections"]))
   

In [ ]:
cor

In [ ]:
lowcor=[]
for key in cor.keys():
  if cor[key]<0.01:
    lowcor.append(key)

In [ ]:
lowcor

In [ ]:
### from now :we would start analysing correlation and set a threshold and drop columns with low correlation than the threshold

In [ ]:

# ypred=clf.predict(xtest)

In [ ]:
# yp=pd.DataFrame()
# yp.insert(0,"Expected",ypred)
# yp.insert(0,"Id",xtest["MachineIdentifier"])

In [ ]:
# yp.head(5)

In [ ]:
# yp.to_csv("out.csv")

In [ ]:
# import xgboost as xgb
# model = xgb.XGBClassifier()
# model.fit(dfm1, ytrain)

In [ ]:
# model.score(X, y)

In [ ]:
# ypred=clf.predict(xtest)


In [ ]:
# yp=pd.DataFrame()
# yp.insert(0,"Expected",ypred)
# yp.insert(0,"Id",xtest["MachineIdentifier"])

In [ ]:
# yp.to_csv("out1.csv")

In [ ]:
# dfm1.to_csv("df_num_mode")

In [ ]:
dfm1.shape

In [ ]:
# X=dfm1
# y=ytrain
# import xgboost as xgb
# model = xgb.XGBClassifier()
# model.fit(X, y)
# model.score(X,y

In [ ]:
# from sklearn.ensemble import HistGradientBoostingClassifier
# X=dfm1
# y=ytrain
# clf = HistGradientBoostingClassifier().fit(X, y)
# clf.score(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
import xgboost as xgb


In [ ]:
X=dfm1.drop(columns=["HasDetections"])
y=dfm1["HasDetections"]

In [ ]:
xt, xvalid, yt, yvalid = train_test_split(X, y, random_state=42, test_size=0.3)

In [ ]:
clf_xgb = xgb.XGBClassifier(learning_rate=0.1, 
                            n_estimators=2000, 
                            max_depth=10,
                            min_child_weight=5,
                            gamma=0,
                            subsample=0.9,
                            colsample_bytree=0.6,
                            objective= 'binary:logistic',
                            nthread=-1,
                            scale_pos_weight=1,
                            reg_alpha = 0.5,
                            reg_lambda = 2,
                            seed=42)


In [ ]:
clf_xgb.fit(xt, yt, eval_set=[(xt, yt), (xvalid, yvalid)], 
early_stopping_rounds=100, eval_metric='auc', verbose=100)
predictions = clf_xgb.predict(xvalid)

In [ ]:
print("accuracy_score", accuracy_score(yvalid, predictions))

In [ ]:
import pickle
filename = 'umairxgb.sav'
pickle.dump(clf_xgb, open(filename, 'wb'))

In [ ]:
le=LabelEncoder()
xt=xtest.select_dtypes(include='object')
for col in xt.columns:
     xtest[col]=le.fit_transform(xtest[col])

In [ ]:
ytestpred=clf_xgb.predict(xtest)

In [ ]:
ytestp=pd.DataFrame()

ytestp.insert(0,"Id",xtest["MachineIdentifier"])


In [ ]:
ytestp.insert(1,"Expected",ytestpred)

In [ ]:
ytestp.head(5)

In [ ]:
ytestp.to_csv("out.csv",index=False)